In [1]:
import networkx as nx
import seaborn as sns
import pandas as pd
import glob
import ast

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
filepath = r"C:\Users\Proprietario\Desktop\SLAPP\SLAPP3-master\6 objectSwarmObserverAgents_AESOP_turtleLib_NetworkX\Analisi 2" # use your path
all_files = glob.glob(filepath + "/*.txt")
G = {}
j = 0

for filename in all_files:
    G[j]=nx.DiGraph()
    fh=open(filename,'r')
    for i, line in enumerate(fh):
        s=line.strip()
        s=s.strip(";").split()
        if (i>6) and (s[0] != 'label') and (s[0] != '}') and (s[1] != '[style=filled,'):
            origin=int(s[0])
            dest=int(s[2])
            G[j].add_edge(origin,dest)
    fh.close()
    j += 1

In [3]:
for i in range (len(G)):
    print('The network has', len(G[i]), 'nodes and', len(G[i].edges()), 'links.')

The network has 100000 nodes and 2810237 links.


In [4]:
for i in range (len(G)):
    print("Is the graph simply connected?", nx.is_strongly_connected(G[i]))

Is the graph simply connected? False


### Show the components of the graph

In [5]:
for i in range (len(G)):
    print("The graph has", nx.number_strongly_connected_components(G[i]),"connected components")

The graph has 79249 connected components


### Extract the largest Connected Component as a subgraph

In [6]:
len(G)

1

In [7]:
graphs = {}

for i in range (len(G)):
    graphs[i] = []
    for c in nx.strongly_connected_components(G[i]):
        graphs[i].append(G[i].subgraph(c))

In [8]:
H = {}

for i in range (len(G)):
    H[i] = graphs[i][0]

In [9]:
for i in range (len(G)):
    print(len(H[i]))

1


In [10]:
for i in range (len(G)):
    print(len(G[i])-len(H[i]))

99999


In [11]:
for i in range (len(G)):
    print("Check that the graph is now connected")
    print(nx.number_strongly_connected_components(H[i]))

Check that the graph is now connected
1


## Clustering

In [12]:
files = glob.glob(filepath + "/*.csv")
dataset = {}
j = 0

for filename in files:
    dataset[j] = pd.read_csv(filename, encoding="utf8")
    j += 1

In [13]:
for i in range(len(dataset)):
    print(dataset[i].head)

<bound method NDFrame.head of        id_autore agent_type     score
0          84122     voters  0.796969
1          88424     voters  0.716394
2          32118     voters  0.495533
3          88924     voters  0.717739
4          22802     voters  0.448072
...          ...        ...       ...
95991      32528     voters  0.502095
95992      31798     voters  0.532108
95993      22600     voters  0.392871
95994      94366     voters  0.847799
95995       7403     voters  0.168907

[95996 rows x 3 columns]>


In [14]:
dataset[0]['score']

0        0.796969
1        0.716394
2        0.495533
3        0.717739
4        0.448072
           ...   
95991    0.502095
95992    0.532108
95993    0.392871
95994    0.847799
95995    0.168907
Name: score, Length: 95996, dtype: float64

In [19]:
N_nodes = {}
N_edges = {}

for j in range (len(G)):
    N_nodes[str(j)] = [0]*8
    N_edges[str(j)] = [0]*8
    
    for i in range (len(dataset[j])):
        if dataset[j]['score'][i] < 0.1:
            N_nodes[str(j)][0] += 1
            N_edges[str(j)][0] += G[j].out_degree(dataset[j]['id_autore'][i])
        if (dataset[j]['score'][i] < 0.2) and (dataset[j]['score'][i] > 0.1):
            N_nodes[str(j)][1] += 1
            N_edges[str(j)][1] += G[j].out_degree(dataset[j]['id_autore'][i])
        if (dataset[j]['score'][i] < 0.3) and (dataset[j]['score'][i] > 0.2):
            N_nodes[str(j)][2] += 1
            N_edges[str(j)][2] += G[j].out_degree(dataset[j]['id_autore'][i])
        if (dataset[j]['score'][i] < 0.45) and (dataset[j]['score'][i] > 0.3):
            N_nodes[str(j)][3] += 1
            N_edges[str(j)][3] += G[j].out_degree(dataset[j]['id_autore'][i])
        if (dataset[j]['score'][i] < 0.55) and (dataset[j]['score'][i] > 0.45):
            N_nodes[str(j)][4] += 1
            N_edges[str(j)][4] += G[j].out_degree(dataset[j]['id_autore'][i])
        if (dataset[j]['score'][i] < 0.7) and (dataset[j]['score'][i] > 0.55):
            N_nodes[str(j)][5] += 1
            N_edges[str(j)][5] += G[j].out_degree(dataset[j]['id_autore'][i])
        if (dataset[j]['score'][i] < 0.8) and (dataset[j]['score'][i] > 0.7):
            N_nodes[str(j)][6] += 1
            N_edges[str(j)][6] += G[j].out_degree(dataset[j]['id_autore'][i])
        if (dataset[j]['score'][i] < 0.9) and (dataset[j]['score'][i] > 0.8):
            N_nodes[str(j)][7] += 1
            N_edges[str(j)][7] += G[j].out_degree(dataset[j]['id_autore'][i])

In [20]:
K_mean = {}
voters_type = 8

for j in range (len(G)):
    K_mean[str(j)] = []
    print('\n')
    for i in range (voters_type):
        K_mean[str(j)].append(N_edges[str(j)][i]/N_nodes[str(j)][i])
        print("The ", i+1, " K_mean value is: ",K_mean[str(j)][i])



The  1  K_mean value is:  33.0248
The  2  K_mean value is:  8.087629640763565
The  3  K_mean value is:  12.553903345724907
The  4  K_mean value is:  12.505305709954522
The  5  K_mean value is:  20.49073748902546
The  6  K_mean value is:  28.73670259146873
The  7  K_mean value is:  42.36681759379043
The  8  K_mean value is:  60.10514579759863
